In [ ]:
!pip install gensim
!pip install -upgrade numpy

import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
from collections import Counter
import math

# Load your preprocessed CSV file
df = pd.read_csv('labelled_news_sentiment.csv')

# Tokenize the processed text
df['tokens'] = df['processed_text'].apply(lambda x: x.split())

print("Sample tokens:", df['tokens'].head())

# Prepare the corpus: a list of token lists
corpus = df['tokens'].tolist()

# Train the Word2Vec model
w2v_model = Word2Vec(sentences=corpus,
                     vector_size=500,    # Dimension of the word vectors
                     window=10,           # Context window size
                     min_count=1,        # Minimum frequency for a word to be considered
                     workers=10,          # Number of threads for training
                     epochs=30)

print("Word2Vec model trained.")

def document_vector(tokens, model):
    # Filter tokens to include only those present in the model's vocabulary
    valid_tokens = [token for token in tokens if token in model.wv]
    if valid_tokens:
        # Compute the mean of the word vectors
        return np.mean(model.wv[valid_tokens], axis=0)
    else:
        # If no valid tokens, return a zero vector
        return np.zeros(model.vector_size)

# Generate document vectors
doc_vectors = np.array([document_vector(tokens, w2v_model) for tokens in df['tokens']])


print("Document vectors shape:", doc_vectors.shape)

X = doc_vectors
y = df['news_sentiment'].values